<a href="https://colab.research.google.com/github/boyerb/Investments/blob/master/Ex09-AlphaBeta_WRDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Investments: Theory, Fundamental Analysis, and Data Driven Analytics**, Bates, Boyer, and Fletcher

# Example Chapter 9: Estimating Alpha Using Data from Alpha Vantage API  
  
In this example we estimate the alpha and beta of Johnson and Johnson (`ticker:JNJ`) relative to the market portfolio along with 95% confidence intervals. In this example, data for Johnson and Johnson returns come from the Alpha Vantage API. Data for the market and short-term T-bill returns come from [Ken French's website](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html). An introduction to the Alpha Vantage API is given in [EX07-Alpha_Vantage.ipynb](https://colab.research.google.com/github/boyerb/Investments/blob/master/Ex07-Alpha_Vantage.ipynb).  

### Imports and Setup
We first import the `simple_finance`, `requests`, `pandas`, and `statsmodels` packages. To improve readability when printing a DataFrame to the consol, we then configure Pandas display options: (1) show all columns without truncation, (2) widen the output to 1000 characters so rows print on a single line, and (3) truncate long tables.


In [ ]:
# Import the simple_finance.py package and avoid using a stale cached version.
!curl -s -O https://raw.githubusercontent.com/boyerb/Investments/master/functions/simple_finance.py
import importlib, simple_finance as sf
importlib.reload(sf)

import requests
import pandas as pd
import statsmodels.api as sm

pd.set_option('display.max_columns', None)   # Show all columns without truncation
pd.set_option('display.width', 1000)   # Set the display width so output stays on one line
pd.set_option("display.max_rows", 20) # Force truncation if DataFrame has more than 20 rows

### Load in Data for Johnson & Johnson Using the Alpha Vantage API
Replace `your_key` in the script with your actual API key. If you don't supply your own API key, the code will still run, but the number of daily requests will be **limited by IP address**.   We then create returns as a percent change in the adjusted close to account for splits, dividends, and other corporate events.

In [ ]:
# Replace the text 'your_key' with your own APPI key
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY_ADJUSTED&symbol=JNJ&apikey=your_key'

# Send an HTTP GET request to the API endpoint and format into a DataFrame
r = requests.get(url)
df = sf.format_alpha_vantage(r)

# Create returns column using Adjusted Close
df["Return"] = df["Adjusted Close"].pct_change()
df = df.dropna()  # drop row with missing value
print(df)

### Load Returns for the Market and Short-Term T-Bills
This function retrieves monthly data for the Fama–French Five-Factor model from [Ken French's website](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html). From the resulting DataFrame, we will extract the market’s excess return `Mkt-RF` and the return on short-term T-bills `RF`.


In [ ]:
FF = sf.get_ff5(start_date="2010-01", end_date="2024-12")
print(FF)

### Merge Two Data Sources
In this code we merge the data from Alpha Vantage and Ken French's website together. We then create the excess return for Johnson & Johnson.

In [ ]:
merged = df.merge(FF, left_index=True, right_index=True, how="inner")
merged = merged[["Return", "Mkt-RF", "RF"]]
merged["Ret-RF"] = merged["Return"] - merged["RF"]
print(merged)

### Estimate Alpha, Beta, and 95% Confidence Intervals
Here we estimate the alpha and beta of Johnson & Johnson relative to the market portfolio along with the 95% confidence intervals.

In [ ]:
X=merged['Mkt-RF']
Y=merged['Ret-RF']
X=sm.add_constant(X)  # we specify that we want to add a constant to the regression equation
model=sm.OLS(Y,X).fit() # run the regression
params = model.params # parameter values
conf_int = model.conf_int(alpha=0.05)  # 95% CIs

# create a table of output
results_df = pd.DataFrame({
    'Parameter': params.index, # 1st column: parameter name
    'Estimate': params.values, # 2nd column: parameer value
    'CI Lower': conf_int[0].values, # 3rd column: lower bound on 95\% CI
    'CI Upper': conf_int[1].values  # 4th column: upper bound on 95\% CI
})
print(results_df)